<a href="https://colab.research.google.com/github/ijohns29/Comparing-regional-Covid-19-data-trends-in-MD-and-MT-to-assess-short-term-risks-of-travel/blob/main/2020_11_17_NYT_Covid_19_data_IJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Looking at recent Covid-19 trends in Maryland and Montana at state level and at more granular level using multiple open data sources



*  Change in daily incidence over past two weeks
*   Change in hospitalizations over past two weeks
* Change in positivity rate as function of age group for MD



# Import Libraries

In [39]:
# Install visualization package
!pip install plotly

In [4]:
# import libraries
# for data analysis: 1 is for 'pandas' i.e. panel data
import pandas as pd # imports panel data. Later in code can just say use pd rather than use pandas
import numpy as np
# for visualization
import plotly.express as px
# to help export files
from google.colab import files

# Import data

In [119]:
# import data e.g. bring in csv. Can't open them here but can take it and project it into notebook. This projection is data frame (df)_relevant_name
# One equals is to define variable , two is for equation 
# Convention is df = pd.read_csv("github link")
# Creates data frame using pandas (prev imported as pd). Read_csv is code under pandas that allows reading of csv file
# Import NYT Covid-19 state level data - 2020-11-17 is the name but autoupdates to latest upon initialization
df_NYT_Covid_19_states_20201127 = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")
# Import NYT Covid-19 county level data - 2020-11-17 is the name but auto updates to latest upon inititalization
df_NYT_Covid_19_counties_20201127 = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
# Import Maryland.gov Covid-19 testing data
df_MD_HHS = pd.read_csv("https://opendata.maryland.gov/resource/avum-x5ua.csv")

# Preview data

In [6]:
# preview data - do this by calling data frame.head(#) If no number default is 5. n = number of rows of data can preview
df_NYT_Covid_19_states_20201127.head(6)


,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
5,2020-01-25,California,6,1,0


In [7]:
# Preview Covid-19 County data
df_NYT_Covid_19_counties_20201127.head(6)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
5,2020-01-25,Orange,California,6059.0,1,0.0


In [9]:
# preview end of county data
df_NYT_Covid_19_counties_20201127.tail(5)

,date,county,state,fips,cases,deaths
784208,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6.0
784209,2020-11-30,Teton,Wyoming,56039.0,1644,2.0
784210,2020-11-30,Uinta,Wyoming,56041.0,1070,4.0
784211,2020-11-30,Washakie,Wyoming,56043.0,440,8.0
784212,2020-11-30,Weston,Wyoming,56045.0,397,1.0


In [71]:
# preview end of state level data 
df_NYT_Covid_19_states_20201127.tail(6)

,date,state,fips,cases,deaths
14858,2020-11-27,Virgin Islands,78,1521,23
14859,2020-11-27,Virginia,51,230444,4044
14860,2020-11-27,Washington,53,165230,2828
14861,2020-11-27,West Virginia,54,45046,712
14862,2020-11-27,Wisconsin,55,399557,3444
14863,2020-11-27,Wyoming,56,31773,215


# Maryland HHS state data
## Cleaning up data and creating appropriate dataframe for it

In [120]:
# Preview MD HHS data
df_MD_HHS.head(10)

,objectid,reportdate,under35,over35,testingvolume
0,1,2020-03-23T10:00:00.000,NaN,NaN,955
1,2,2020-03-24T10:00:00.000,NaN,NaN,1120
2,3,2020-03-25T10:00:00.000,NaN,NaN,3340
3,4,2020-03-26T10:00:00.000,NaN,NaN,1820
4,5,2020-03-27T10:00:00.000,NaN,NaN,3075
5,6,2020-03-28T10:00:00.000,NaN,NaN,1308
6,7,2020-03-29T10:00:00.000,9.56,12.00,1585
7,8,2020-03-30T10:00:00.000,10.06,13.34,2054
8,9,2020-03-31T10:00:00.000,11.03,14.76,2947
9,10,2020-04-01T10:00:00.000,11.52,15.70,2538


In [121]:
# Preview MD HHS data
df_MD_HHS.tail(10)

,objectid,reportdate,under35,over35,testingvolume
243,244,2020-11-21T10:00:00.000,7.03,6.67,49922
244,245,2020-11-22T10:00:00.000,7.11,6.73,31242
245,246,2020-11-23T10:00:00.000,6.79,6.48,29162
246,247,2020-11-24T10:00:00.000,6.80,6.34,45495
247,248,2020-11-25T10:00:00.000,6.65,6.20,41777
248,249,2020-11-26T10:00:00.000,6.57,6.12,47304
249,250,2020-11-27T10:00:00.000,6.69,6.18,27706
250,251,2020-11-28T10:00:00.000,6.87,6.31,39249
251,252,2020-11-29T10:00:00.000,7.25,6.62,27350
252,253,2020-11-30T10:00:00.000,7.75,7.08,30697


In [87]:
# Create data frame to test code with. df.shape returns number of rows (i) and columns (0)
df_edit_MD_HHS = df_MD_HHS
df_edit_MD_HHS.tail()
df_edit_MD_HHS.shape

(253, 5)

In [122]:
# Creates blank list for split data to be stored in e.g. df_split_MD_HHS
# Uses for loop to look over entire column (date column '1') and splits the data at the T point. [0] after indicates keep the first part of the split
df_split_MD_HHS = []
for i in range(df_edit_MD_HHS.shape[0]):
  df_split_MD_HHS.append(df_edit_MD_HHS.iloc[i,1].split('T')[0])  

In [123]:
# Converts list to dataframe
dfObj = pd.DataFrame(df_split_MD_HHS, columns = ['date'])
#preview newly created dataframe
dfObj.head()

,date
0,2020-03-23
1,2020-03-24
2,2020-03-25
3,2020-03-26
4,2020-03-27


In [124]:
# Fill report date with split date info from dfObj dataframe
df_MD_HHS["reportdate"] = dfObj["date"]
df_MD_HHS.head()

,objectid,reportdate,under35,over35,testingvolume
0,1,2020-03-23,NaN,NaN,955
1,2,2020-03-24,NaN,NaN,1120
2,3,2020-03-25,NaN,NaN,3340
3,4,2020-03-26,NaN,NaN,1820
4,5,2020-03-27,NaN,NaN,3075


In [125]:
# Rename column headings so can be merged with other data sets if desired
df_MD_HHS = df_MD_HHS.rename(columns={"reportdate" : "date",
                                "under35": "% pos. under 35 yrs",
                               "over35": "% pos. over 35 yrs"})
df_MD_HHS.head()

,objectid,date,% pos. under 35 yrs,% pos. over 35 yrs,testingvolume
0,1,2020-03-23,NaN,NaN,955
1,2,2020-03-24,NaN,NaN,1120
2,3,2020-03-25,NaN,NaN,3340
3,4,2020-03-26,NaN,NaN,1820
4,5,2020-03-27,NaN,NaN,3075


In [127]:
df_MD_HHS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   objectid             253 non-null    int64  
 1   date                 253 non-null    object 
 2   % pos. under 35 yrs  247 non-null    float64
 3   % pos. over 35 yrs   247 non-null    float64
 4   testingvolume        253 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 10.0+ KB


# Data Filtering

In [10]:
# Filter Data
# Look at all of the column names in the state data set. This is useful to do prior to creating new dataset so can pull exact name in order to select appropriate column
#Format is df_name.columns
df_NYT_Covid_19_states_20201127.columns

Index(['date', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [72]:
# Look at all of the column names in the county level data 
df_NYT_Covid_19_counties_20201127.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [49]:
# Look at all of the column names in the MD HHS data
df_MD_HHS.columns

Index(['objectid', 'reportdate', 'under35', 'over35', 'testingvolume'], dtype='object')

In [51]:
# Look at object types of data set
df_MD_HHS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   objectid       252 non-null    int64  
 1   reportdate     252 non-null    object 
 2   under35        246 non-null    float64
 3   over35         246 non-null    float64
 4   testingvolume  252 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 10.0+ KB


In [19]:
#look at unique values in columns -> useful for grabbing exact value searching for e.g. 'Maryland' with data repository spelling
# Format is df_name["column name of interest"].unique()
df_NYT_Covid_19_states_20201127["state"].unique()

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands'], dtype=object)

In [73]:
# look at unique names in columns for county level data
df_NYT_Covid_19_counties_20201127["county"].unique()

array(['Snohomish', 'Cook', 'Orange', ..., 'Skagway Municipality',
       'Esmeralda', 'Loving'], dtype=object)

In [32]:
#df_name["column heading of interest"].unique - pulls entire list with repeats as well included, basically full printout of column
df_NYT_Covid_19_states_20201127["state"].unique

<bound method Series.unique of 0           Washington
1           Washington
2           Washington
3             Illinois
4           Washington
             ...      
14859         Virginia
14860       Washington
14861    West Virginia
14862        Wisconsin
14863          Wyoming
Name: state, Length: 14864, dtype: object>

In [14]:
# df_name[""].nunique() - gives total number of unique state + territories listed
df_NYT_Covid_19_states_20201127["state"].nunique()

55

In [15]:
df_NYT_Covid_19_Maryland_20201127.head(5)

,date,state,fips,cases,deaths
307,2020-03-05,Maryland,24,3,0
331,2020-03-06,Maryland,24,3,0
361,2020-03-07,Maryland,24,3,0
396,2020-03-08,Maryland,24,5,0
432,2020-03-09,Maryland,24,6,0


In [34]:
# df_name.info() - gives high level data about data frame that is called e.g. # columns, column name, information types
df_NYT_Covid_19_Maryland_20201127.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 307 to 14830
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    268 non-null    object
 1   state   268 non-null    object
 2   fips    268 non-null    int64 
 3   cases   268 non-null    int64 
 4   deaths  268 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 12.6+ KB


In [35]:
# df_name.describe() - general descriptive statistics, does not pull data for all columns only those that are coded as numbers e.g. int64
df_NYT_Covid_19_Maryland_20201127.describe()

,fips,cases,deaths
count,268.0,268.000000,268.000000
mean,24.0,78745.134328,2742.014925
std,0.0,52722.471567,1479.057173
min,24.0,3.000000,0.000000
25%,24.0,33285.500000,1673.250000
50%,24.0,76589.000000,3353.000000
75%,24.0,121153.250000,3886.000000
max,24.0,193205.000000,4569.000000


# Creating new data frames from imported data

In [8]:
#Pull out a column from NYT orig dataframe (df) to new dataframe with data repository names verified as above - pulls out Maryland data
df_NYT_Covid_19_Maryland_20201127 = df_NYT_Covid_19_states_20201127[df_NYT_Covid_19_states_20201127["state"]== "Maryland"]

In [13]:
#preview new dataframe containing only MD data
df_NYT_Covid_19_Maryland_20201127.head()

,date,state,fips,cases,deaths
307,2020-03-05,Maryland,24,3,0
331,2020-03-06,Maryland,24,3,0
361,2020-03-07,Maryland,24,3,0
396,2020-03-08,Maryland,24,5,0
432,2020-03-09,Maryland,24,6,0


In [9]:
# Pull out Montana Data from state level data 
df_NYT_Covid_19_Montana_20201127 = df_NYT_Covid_19_states_20201127[df_NYT_Covid_19_states_20201127["state"]== "Montana"]
df_NYT_Covid_19_Montana_20201127.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 612 to 15056
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    264 non-null    object
 1   state   264 non-null    object
 2   fips    264 non-null    int64 
 3   cases   264 non-null    int64 
 4   deaths  264 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 12.4+ KB


In [28]:
# preview MT data
df_NYT_Covid_19_Montana_20201127.tail()

,date,state,fips,cases,deaths
14836,2020-11-27,Montana,30,59796,658
14891,2020-11-28,Montana,30,60845,670
14946,2020-11-29,Montana,30,61801,672
15001,2020-11-30,Montana,30,62256,684
15056,2020-12-01,Montana,30,63324,700


# Merging and editing dataframes

In [29]:
df_new_index_MT = df_NYT_Covid_19_Montana_20201127.set_index('date')

In [33]:
df_new_index_MD = df_NYT_Covid_19_Maryland_20201127.set_index('date')

In [133]:
df_daily_case_death_MT = df_new_index_MT[{'cases', 'deaths'}].diff()
df_daily_case_death_MT.tail()
# Merge MT data
df_MT_daily_cumulative_merge = pd.merge(df_NYT_Covid_19_Montana_20201127,
                                 df_daily_case_death_MT,
                                 how = "right",
                                 on = "date")
df_MT_daily_cumulative_merge.head()
# Rename columns
df_MT_daily_cumulative_merge = df_MT_daily_cumulative_merge.rename(columns={"state" : "state_MT",
                                "fips": "fips_MT",
                               "cases_x": "cases_MT",
                               "deaths_x": "deaths_MT",
                               "cases_y": "daily cases_MT",
                               "deaths_y": "daily deaths_MT"})
df_MT_daily_cumulative_merge.head()

,date,state_MT,fips_MT,cases_MT,deaths_MT,daily cases_MT,daily deaths_MT
0,2020-03-13,Montana,30,4,0,NaN,NaN
1,2020-03-14,Montana,30,6,0,2.0,0.0
2,2020-03-15,Montana,30,6,0,0.0,0.0
3,2020-03-16,Montana,30,8,0,2.0,0.0
4,2020-03-17,Montana,30,8,0,0.0,0.0


In [35]:
df_daily_case_death_MD = df_new_index_MD[{'cases', 'deaths'}].diff()
df_daily_case_death_MD.tail()

,cases,deaths
date,,
2020-11-27,2570.0,22.0
2020-11-28,1398.0,33.0
2020-11-29,2124.0,23.0
2020-11-30,1983.0,16.0
2020-12-01,2769.0,32.0


In [53]:
# Merge Maryland dataframes
df_MD_HHS_state_merge = pd.merge(df_NYT_Covid_19_Maryland_20201127,
                                 df_daily_case_death_MD,
                                 how = "right",
                                 on = "date")
# Preview merged dataframe 
df_MD_HHS_state_merge.head()

,date,state,fips,cases_x,deaths_x,cases_y,deaths_y
0,2020-03-05,Maryland,24,3,0,NaN,NaN
1,2020-03-06,Maryland,24,3,0,0.0,0.0
2,2020-03-07,Maryland,24,3,0,0.0,0.0
3,2020-03-08,Maryland,24,5,0,2.0,0.0
4,2020-03-09,Maryland,24,6,0,1.0,0.0


In [55]:
# Rename columns of data frame
df_MD_HHS_state_merge = df_MD_HHS_state_merge.rename(columns={"state" : "state_MD",
                                "fips": "fips_MD",
                               "cases_x": "cases_MD",
                               "deaths_x": "deaths_MD",
                               "cases_y": "daily cases_MD",
                               "deaths_y": "daily deaths_MD"})
# Preview renamed data
df_MD_HHS_state_merge.tail()

,date,state_MD,fips_MD,cases_MD,deaths_MD,daily cases_MD,daily deaths_MD
267,2020-11-27,Maryland,24,193205,4569,2570.0,22.0
268,2020-11-28,Maryland,24,194603,4602,1398.0,33.0
269,2020-11-29,Maryland,24,196727,4625,2124.0,23.0
270,2020-11-30,Maryland,24,198710,4641,1983.0,16.0
271,2020-12-01,Maryland,24,201479,4673,2769.0,32.0


In [130]:
# Merge Maryland and filtered HHS data
df_MD_HHS_filter_state_merge = pd.merge(df_MD_HHS_state_merge,
                                 df_MD_HHS[{("date"), ("% pos. under 35 yrs"), ("% pos. over 35 yrs"), ("testingvolume")}],
                                 how = "right",
                                 on = "date")
df_MD_HHS_filter_state_merge.head()

,date,state_MD,fips_MD,cases_MD,deaths_MD,daily cases_MD,daily deaths_MD,% pos. under 35 yrs,testingvolume,% pos. over 35 yrs
0,2020-03-23,Maryland,24,290,3,45.0,0.0,NaN,955,NaN
1,2020-03-24,Maryland,24,349,4,59.0,1.0,NaN,1120,NaN
2,2020-03-25,Maryland,24,423,4,74.0,0.0,NaN,3340,NaN
3,2020-03-26,Maryland,24,581,4,158.0,0.0,NaN,1820,NaN
4,2020-03-27,Maryland,24,790,5,209.0,1.0,NaN,3075,NaN


In [135]:
# Full merge of Maryland and Montana Data Frames
df_MD_MT_grand_state_merge = pd.merge(df_MD_HHS_filter_state_merge,
                                 df_MT_daily_cumulative_merge,
                                 how = "left",
                                 on = "date")
# Preview data
df_MD_MT_grand_state_merge.head()

,date,state_MD,fips_MD,cases_MD,deaths_MD,daily cases_MD,daily deaths_MD,% pos. under 35 yrs,testingvolume,% pos. over 35 yrs,state_MT,fips_MT,cases_MT,deaths_MT,daily cases_MT,daily deaths_MT
0,2020-03-23,Maryland,24,290,3,45.0,0.0,NaN,955,NaN,Montana,30,45,0,11.0,0.0
1,2020-03-24,Maryland,24,349,4,59.0,1.0,NaN,1120,NaN,Montana,30,51,0,6.0,0.0
2,2020-03-25,Maryland,24,423,4,74.0,0.0,NaN,3340,NaN,Montana,30,65,0,14.0,0.0
3,2020-03-26,Maryland,24,581,4,158.0,0.0,NaN,1820,NaN,Montana,30,90,0,25.0,0.0
4,2020-03-27,Maryland,24,790,5,209.0,1.0,NaN,3075,NaN,Montana,30,121,1,31.0,1.0


In [136]:
# Select specific time frame from Maryland data set e.g. last 30 days
df_MD_MT_grand_state_merge_30days = df_MD_MT_grand_state_merge.tail(30)
#Preview data
df_MD_MT_grand_state_merge_30days.head()

,date,state_MD,fips_MD,cases_MD,deaths_MD,daily cases_MD,daily deaths_MD,% pos. under 35 yrs,testingvolume,% pos. over 35 yrs,state_MT,fips_MT,cases_MT,deaths_MT,daily cases_MT,daily deaths_MT
223,2020-11-01,Maryland,24,146627,4152,1034.0,5.0,4.49,26814,3.64,Montana,30,33495,378,694.0,1.0
224,2020-11-02,Maryland,24,147477,4155,850.0,3.0,4.47,19886,3.64,Montana,30,34317,387,822.0,9.0
225,2020-11-03,Maryland,24,148249,4162,772.0,7.0,4.68,25270,3.78,Montana,30,35222,400,905.0,13.0
226,2020-11-04,Maryland,24,149232,4172,983.0,10.0,4.81,27944,3.87,Montana,30,36012,404,790.0,4.0
227,2020-11-05,Maryland,24,150424,4182,1192.0,10.0,5.07,35378,3.99,Montana,30,37024,409,1012.0,5.0


# Some test code

In [37]:
#Select specific time frame from Maryland data set e.g. last 30 days
df_NYT_Covid_19_Maryland_20201127_30days = df_NYT_Covid_19_Maryland_20201127.tail(30)

In [38]:
# preview last 30 days of data for MD
df_NYT_Covid_19_Maryland_20201127_30days.head()

,date,state,fips,cases,deaths
13455,2020-11-02,Maryland,24,147477,4155
13510,2020-11-03,Maryland,24,148249,4162
13565,2020-11-04,Maryland,24,149232,4172
13620,2020-11-05,Maryland,24,150424,4182
13675,2020-11-06,Maryland,24,151964,4194


In [137]:
#df_NYT_Covid_19_Maryland_20201127_November2020 = df_NYT_Covid_19_Maryland_20201127[df_NYT_Covid_19_Maryland_20201127.tail(30)

In [59]:
#data frame with double filter e..g filter to only Maryland and to only ~ last month of data
#df_NYT_Covid_19_Maryland_20201127_November2020 = df_NYT_Covid_19_states_20201127[(df_NYT_Covid_19_states_20201127["state"] == "Maryland") & (df_NYT_Covid_19_states_20201127["date"] == "2020-11-01 to 2020-11-30")]

In [60]:
# preview double filter data containing Maryland specific data and date specific data
#df_NYT_Covid_19_Maryland_20201127_November2020.head()

,date,state,fips,cases,deaths


# Merging practice

In [20]:
# Merging Maryland and Montana state level data, ONLY works under two conditions when use how = right and date and when use how = left and date
# When use how = left and on = date the right side column lists number with decimal formatting. Left means merge data from right dataset onto left dataset
# Have to use date because its the only column with same matching key AND same  data contained down the rows
# Can do df_NYT_Covid_19-Montana_20201127[("desired column"), ("other desired column")] to specify what data gets merged from right data set onto left data set
df_MD_MT_merge = pd.merge(df_NYT_Covid_19_Maryland_20201127,
                          df_NYT_Covid_19_Montana_20201127, 
                          how = "right",
                          on = "date")

In [21]:
# preview merged MD_MT data
df_MD_MT_merge.tail(5)

,date,state_x,fips_x,cases_x,deaths_x,state_y,fips_y,cases_y,deaths_y
258,2020-11-26,Maryland,24,190635,4547,Montana,30,59682,658
259,2020-11-27,Maryland,24,193205,4569,Montana,30,59796,658
260,2020-11-28,Maryland,24,194603,4602,Montana,30,60845,670
261,2020-11-29,Maryland,24,196727,4625,Montana,30,61801,672
262,2020-11-30,Maryland,24,198710,4641,Montana,30,62256,684


In [23]:
# Rename state_x, fips_x, cases_x, deaths_x, state_y, fips_y, cases_y, deaths_y to state specific columns
# Merge data from right onto left sooo Left is maryland right is MT
# format is df_name.rename(columns={"old name" : "new name"}) Columns specifies renaming column name
# Overwrite of original data frame with more specific names
df_MD_MT_merge = df_MD_MT_merge.rename(columns={"state_x" : "state_MD",
                                "fips_x": "fips_MD",
                               "cases_x": "cases_MD",
                               "deaths_x": "deaths_MD",
                               "state_y": "state_MT",
                               "fips_y": "fips_MT",
                               "cases_y": "cases_MT",
                               "deaths_y": "deaths_MT"})

In [24]:
# Call of changed data frame to see changes
df_MD_MT_merge.head()

,date,state_MD,fips_MD,cases_MD,deaths_MD,state_MT,fips_MT,cases_MT,deaths_MT
0,2020-03-13,Maryland,24,18,0,Montana,30,4,0
1,2020-03-14,Maryland,24,27,0,Montana,30,6,0
2,2020-03-15,Maryland,24,32,0,Montana,30,6,0
3,2020-03-16,Maryland,24,39,0,Montana,30,8,0
4,2020-03-17,Maryland,24,57,0,Montana,30,8,0


In [131]:
# Calculate new daily cases and new daily deaths for each state using current day - previous day formulation e.g. daily new cases = current day - previous day
# data in dataframes are classified as integers - no need to change 
# If desire to redefine at later data df_MD_MT_merge["cases_MD"] = df_MD_MT_merge["cases_MD"].astype(float, int, etc)
#df_MD_MT_merge.diff(periods=0,axis=0)
#df["daily cases_MD"] = (df_MD_MT_merge["cases_MD"] - df_MD_MT_merge["cases_MD"])
# to create new column in data set with unique name form is as follows: must be of same type e.g. both int, floats, objects etc
# df_xx["unique name"] = df_xx["col name" + df_xx["col name"]]


# Creating visualizations

In [138]:
df_MD_MT_grand_state_merge_30days.head()

,date,state_MD,fips_MD,cases_MD,deaths_MD,daily cases_MD,daily deaths_MD,% pos. under 35 yrs,testingvolume,% pos. over 35 yrs,state_MT,fips_MT,cases_MT,deaths_MT,daily cases_MT,daily deaths_MT
223,2020-11-01,Maryland,24,146627,4152,1034.0,5.0,4.49,26814,3.64,Montana,30,33495,378,694.0,1.0
224,2020-11-02,Maryland,24,147477,4155,850.0,3.0,4.47,19886,3.64,Montana,30,34317,387,822.0,9.0
225,2020-11-03,Maryland,24,148249,4162,772.0,7.0,4.68,25270,3.78,Montana,30,35222,400,905.0,13.0
226,2020-11-04,Maryland,24,149232,4172,983.0,10.0,4.81,27944,3.87,Montana,30,36012,404,790.0,4.0
227,2020-11-05,Maryland,24,150424,4182,1192.0,10.0,5.07,35378,3.99,Montana,30,37024,409,1012.0,5.0


In [140]:
df_MD_MT_grand_state_merge_30days.columns

Index(['date', 'state_MD', 'fips_MD', 'cases_MD', 'deaths_MD',
       'daily cases_MD', 'daily deaths_MD', '% pos. under 35 yrs',
       'testingvolume', '% pos. over 35 yrs', 'state_MT', 'fips_MT',
       'cases_MT', 'deaths_MT', 'daily cases_MT', 'daily deaths_MT'],
      dtype='object')

In [141]:
# Create bar graph visualization of daily covid cases
bar_daily_cases_MD = px.bar(df_MD_MT_grand_state_merge_30days,x="date", y="daily cases_MD", title = "Daily cases of Covid-19 in Maryland")

In [142]:
bar_daily_cases_MD

In [ ]:
# daily cases MT